In [2]:
# Imports
from lib.utils import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.nn.functional import relu,one_hot
from tqdm import tqdm
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
device = 'cuda'
torch.cuda.empty_cache()

/home/andrew/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
X,y = load_raw_list([21])
y = one_hot(y,num_classes=3).reshape(-1,3).float()
X = X.reshape(-1,5000)
dataset = TensorDataset(X,y)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

In [4]:
class MLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.fc1 = nn.Linear(in_features=5000,out_features=100)
        self.fc2 = nn.Linear(in_features=100,out_features=3)

    def forward(self,x):
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        
        return x
model = MLP().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
training_losses = []

In [5]:
# Get initial loss
total_loss = 0
model.train()
for (X,y) in dataloader:
    X,y = X.cuda(), y.cuda()
    logits = model(X)
    loss = criterion(logits,y)
    total_loss += loss.item()
total_loss/len(dataloader)

1.1191053951487822

In [28]:
for epoch in range(1000):
    training_loss = 0
    model.train()
    for (X,y) in dataloader:
        X,y = X.to(device), y.to(device)
        logits = model(X)
        loss = criterion(logits,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
    training_losses.append(training_loss/len(dataloader))
    if(epoch%50 == 0):
        plt.plot(training_losses)
        plt.savefig('loss.jpg')
        plt.close()